In [ ]:
img_folder = '../data/raw/train_20241023/daytime'
bkgr_folder = '../data/processed/bkgr'
bkgr_prefix = 'background_'
output_folder = '../data/processed/obj_bkgr/daytime'

# img_folder = '../data/raw/train_20241023/nighttime'
# bkgr_folder = '../data/processed/bkgr'
# bkgr_prefix = 'background_'
# output_folder = '../data/processed/obj_bkgr/nighttime'

In [ ]:
import glob
import cv2
import os
from tqdm import tqdm
import shutil

# Lấy danh sách ảnh và background
img_paths = glob.glob(os.path.join(img_folder, '*.jpg'))
bkgr_paths = {f"{bkgr_prefix}{str(i).zfill(2)}.jpg": cv2.imread(f"{bkgr_folder}/{bkgr_prefix}{str(i).zfill(2)}.jpg") for i in range(1, 11)}

# Tạo folder output nếu chưa tồn tại
os.makedirs(output_folder, exist_ok=True)

# Duyệt qua từng ảnh trong img_paths
for img_path in tqdm(img_paths, desc="Processing Images"):
    # Lấy tên ảnh và đọc file label
    img_name = os.path.basename(img_path)
    label_path = img_path.replace('.jpg', '.txt')
    img = cv2.imread(img_path)
    
    # Kiểm tra nếu label tồn tại và ảnh được load
    if not os.path.exists(label_path) or img is None:
        continue

    # Đọc nội dung file label
    with open(label_path, 'r') as f:
        labels = []
        for line in f:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            labels.append((class_id, x_center, y_center, width, height))
    
    # Sắp xếp labels theo y_center để vẽ từ trên xuống dưới
    labels.sort(key=lambda x: x[2])  # y_center là phần tử thứ 2

    # Lấy background tương ứng
    cam_id = img_name.split('_')[1].zfill(2)  # lấy XX từ cam_XX_YYYYY.jpg
    bkgr_name = f"{bkgr_prefix}{cam_id}.jpg"
    bkgr_img = bkgr_paths.get(bkgr_name, None)
    
    # Nếu không có background tương ứng, bỏ qua ảnh này
    if bkgr_img is None:
        continue

    # Tạo một bản sao của background để vẽ
    output_img = bkgr_img.copy()
    h, w = img.shape[:2]

    # Vẽ từng object lên background
    for class_id, x_center, y_center, width, height in labels:
        # Tính toạ độ trên ảnh
        x = int((x_center - width / 2) * w)
        y = int((y_center - height / 2) * h)
        w_box = int(width * w)
        h_box = int(height * h)

        # Cắt object từ ảnh gốc và vẽ lên background
        object_img = img[y:y+h_box, x:x+w_box]
        output_img[y:y+h_box, x:x+w_box] = object_img

    # Lưu ảnh đã vẽ vào folder output
    output_path = os.path.join(output_folder, img_name)
    cv2.imwrite(output_path, output_img)
    shutil.copy(label_path, output_path.replace('.jpg', '.txt'))

Processing Images: 100%|██████████| 4629/4629 [00:32<00:00, 143.67it/s]
